In [1]:
import pandas as pd
from datetime import date
import os

In [2]:
tDay=date.today()

In [3]:
quote='quote borrow: 1709-HK / 2598-HK / 1783-HK / 917-HK / 2522-HK / 1280-HK / 2473-HK / 2477-HK / 2490-HK / 2495-HK / 412-HK / 2582-HK / 9636-HK / 7200-HK / 7226-HK / 600418-CH / 600104-CH'

In [4]:
text='''
Huatai HT cover 205.5K shares 601988-C1 limit 5.35, DMA
Huatai HT cover 1.09mn shares 601988-C1 limit 5.38, DMA
Huatai HT cover 650K shares 601988-CH limit 5.53, POV 10% passive 
Huatai short 6mn shares 1114-HK limit 3.88, AQUA 12% passive
Huatai BUY 500K shares 601919-C1 limit 14.7, POV 3% 
CLSA short 588K shares 1280-HK limit 2.54, POV 12% passive 
CLSA short 224K shares 2477-HK limit 22.95, POV 15% passive 
CLSA short 350K shares 2477-HK limit 23.5, POV 15% passive 
MACQ short 980 shares 2495-HK limit 254.0, NEO 10%-16%
JEFS short 94K shares 1783-HK limit 7.0, POV 12% passive
'''

In [5]:
text2=text.replace("  "," ").replace(" shares","").replace(" limit","").replace(",","").replace("CLSA Desk","CLSA_HT").replace("DESK POV","DESK").replace("Desk POV","DESK").replace(" HT","_HT").replace(" (low-touch)","").replace("~","").replace('passive ','passive').replace('MACQ','MEEX').replace('NEO','POV').replace('below ','1-').replace('near touch','neartouch').replace('near-touch ','NearTouch').replace('near-touch','NearTouch').replace('Low','5-10%').replace('Mid','10-20%').replace('High','15-30%').replace('Maybank','MKEA').replace('Inline','POV')
tradelist=[trades.rstrip().split(" ") for trades in text2.split("\n") if trades != ""]
#tradelist

In [6]:
cimblist=[ " ".join(x[1:]) for x in tradelist if "CIMB" in x]
for order in cimblist:
    print(order)

In [7]:
header=['Broker','Side','Quantity','Ticker','Price','Strategy','Vol(%)','Style']

In [8]:
def qty_conv(q):
    if q[-2:].lower() in ['mn','mm'] :
        qty=int(float(q[:-2])*1000000)
    elif q[-1].lower()=='k':
        qty=int(float(q[:-1])*1000)
    else:
        qty=int(q)
    
    return qty

In [9]:
tradeframe=pd.DataFrame(tradelist,columns=header)
#tradeframe=pd.DataFrame(tradelist)
#tradeframe

In [10]:
tradeframe['Ticker']=tradeframe['Ticker'].map(lambda x: x.replace("-"," ")+" EQUITY")
tradeframe['Quantity']=tradeframe['Quantity'].map(qty_conv)
tradeframe['Vol(%)']=tradeframe['Vol(%)'].map(lambda x: x.replace("%","") if x is not None else x)
tradeframe['Side']=tradeframe['Side'].map(lambda x:x.upper())
tradeframe['Side']=tradeframe['Side'].map({'SHORT':'SHRT','COVER':'COVR',"BUY":"BUY","SELL":'SELL'})
#tradeframe

In [11]:
brokers=['CLSA','CLSA_HT','Huatai','Huatai_HT','CICC_HT','CICC','Haitong','Haitong_HT','MEEX','MKEA','JEFS']

In [12]:
tradeframe2=tradeframe[tradeframe['Broker'].isin(brokers)]
#tradeframe2
tradeframe2['Style']=tradeframe2.apply(lambda row: row['Style'].replace('passive','neartouch') if row['Broker']=='MEEX' and row['Style']=='passive' else row['Style'], axis=1)
tradeframe2

,Broker,Side,Quantity,Ticker,Price,Strategy,Vol(%),Style
0,Huatai_HT,COVR,205500,601988 C1 EQUITY,5.35,DMA,None,None
1,Huatai_HT,COVR,1090000,601988 C1 EQUITY,5.38,DMA,None,None
2,Huatai_HT,COVR,650000,601988 CH EQUITY,5.53,POV,10,passive
3,Huatai_HT,SHRT,6000000,1114 HK EQUITY,3.88,AQUA,12,passive
4,Huatai,BUY,500000,601919 C1 EQUITY,14.7,POV,3,None
5,CLSA,SHRT,588000,1280 HK EQUITY,2.54,POV,12,passive
6,CLSA,SHRT,224000,2477 HK EQUITY,22.95,POV,15,passive
7,CLSA,SHRT,350000,2477 HK EQUITY,23.5,POV,15,passive
8,MEEX,SHRT,980,2495 HK EQUITY,254.0,POV,10-16,None
9,JEFS,SHRT,94000,1783 HK EQUITY,7.0,POV,12,passive


In [13]:
tradeframe2.to_csv('trades_{}.csv'.format(tDay.strftime("%Y%m%d")),index=False)

In [14]:
quoteList=quote.replace("Quote borrow: ","").replace("quote borrow: ","").replace(":","").replace("Haitong","HT").replace("Guangfa","GF").split("/ ")
#quoteList

In [15]:
def quote(broker):    
    List=[x for x in quoteList if "(" not in x]+[x.split()[0] for x in quoteList if broker in x]
    String=broker+":\nHi team, could you quote borrow on "
    if broker=='Citi'or broker=='JEFF'or broker=='Tiger':
        List=[x for x in List if 'HK' in x]
    for x in List:
        String=String+x.strip()+", "
    String=String+"tks\n"
    String
    return String

In [16]:
brokerList=['GF','HT','CLSA','CICC','Huatai','MACQ','MB','Citi','JEFF','Tiger']#'CICC',
for broker in brokerList:
    print(quote(broker))

GF:
Hi team, could you quote borrow on 1709-HK, 2598-HK, 1783-HK, 917-HK, 2522-HK, 1280-HK, 2473-HK, 2477-HK, 2490-HK, 2495-HK, 412-HK, 2582-HK, 9636-HK, 7200-HK, 7226-HK, 600418-CH, 600104-CH, tks

HT:
Hi team, could you quote borrow on 1709-HK, 2598-HK, 1783-HK, 917-HK, 2522-HK, 1280-HK, 2473-HK, 2477-HK, 2490-HK, 2495-HK, 412-HK, 2582-HK, 9636-HK, 7200-HK, 7226-HK, 600418-CH, 600104-CH, tks

CLSA:
Hi team, could you quote borrow on 1709-HK, 2598-HK, 1783-HK, 917-HK, 2522-HK, 1280-HK, 2473-HK, 2477-HK, 2490-HK, 2495-HK, 412-HK, 2582-HK, 9636-HK, 7200-HK, 7226-HK, 600418-CH, 600104-CH, tks

CICC:
Hi team, could you quote borrow on 1709-HK, 2598-HK, 1783-HK, 917-HK, 2522-HK, 1280-HK, 2473-HK, 2477-HK, 2490-HK, 2495-HK, 412-HK, 2582-HK, 9636-HK, 7200-HK, 7226-HK, 600418-CH, 600104-CH, tks

Huatai:
Hi team, could you quote borrow on 1709-HK, 2598-HK, 1783-HK, 917-HK, 2522-HK, 1280-HK, 2473-HK, 2477-HK, 2490-HK, 2495-HK, 412-HK, 2582-HK, 9636-HK, 7200-HK, 7226-HK, 600418-CH, 600104-CH, tk

In [17]:
cimbList=[x for x in quoteList if "(" not in x]+[x.split()[0] for x in quoteList if "CIMB" in x]
shares=[1000000]*len(cimbList)
df=pd.DataFrame(shares,cimbList,columns=['qty'])
#csv_path='borrow.csv'
#df.to_csv('borrow.csv')
#os.system(f'start excel "{csv_path}"') 
df

,qty
1709-HK,1000000
2598-HK,1000000
1783-HK,1000000
917-HK,1000000
2522-HK,1000000
1280-HK,1000000
2473-HK,1000000
2477-HK,1000000
2490-HK,1000000
2495-HK,1000000


In [18]:
mbList=[x for x in quoteList if "(" not in x]+[x.split()[0] for x in quoteList if "MB" in x]
shares=[1000000]*len(mbList)
df=pd.DataFrame(shares,mbList,columns=['qty'])
#csv_path='borrow.csv'
#df.to_csv('borrow.csv')
#os.system(f'start excel "{csv_path}"') 
df

,qty
1709-HK,1000000
2598-HK,1000000
1783-HK,1000000
917-HK,1000000
2522-HK,1000000
1280-HK,1000000
2473-HK,1000000
2477-HK,1000000
2490-HK,1000000
2495-HK,1000000


In [19]:
macqList=[x for x in quoteList if "(" not in x]+[x.split()[0] for x in quoteList if 'MACQ' in x]
shares=[1000000]*len(macqList)
macq=pd.DataFrame(shares,macqList,columns=['quantity'])
macq=macq.reset_index()
macq.columns=['ticker','quantity']
def symbol(ticker):
    if ticker[-2:]=="CH":
        if ticker[0]=="6":
            ticker=ticker.replace("CH","SH")
        else:
            ticker=ticker.replace("CH","ZK")
    if ticker[-2:]=="HK":
        ticker=ticker.zfill(7)
    ticker=ticker.replace("-",".")
    return ticker
macq['ticker']=macq['ticker'].map(lambda x: x.replace(" ",""))
macq['ticker']=macq['ticker'].map(lambda x: symbol(x))
macq.loc[macq.shape[0]]=["1211.HK",500000]
macq.loc[macq.shape[0]]=["0300.HK",500000]
macq.loc[macq.shape[0]]=["0921.HK",500000]
csv_path='macqborrow.csv'
macq.to_csv('macqborrow.csv',index=False)
#os.system(f'start excel "{csv_path}"') 
macq

,ticker,quantity
0,1709.HK,1000000
1,2598.HK,1000000
2,1783.HK,1000000
3,0917.HK,1000000
4,2522.HK,1000000
5,1280.HK,1000000
6,2473.HK,1000000
7,2477.HK,1000000
8,2490.HK,1000000
9,2495.HK,1000000
